# Мотивация

Ключом производительности Apache Spark является оптимизатор Catalyst, который умеет находить наиболее оптимальные стратегии исполнения запросов на кластере. Глубокое понимание принципов работы оптимизатора лучше анализировать план запроса, а также понимать почему план так выглядит.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F

# Подготовка

## Spark Контекст

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("catalyst")
        .master("local[4]")
        .getOrCreate()
)
sc = spark.sparkContext

После старта сессии `Spark UI` становится доступным на порту [`4040`](http://localhost:4040)

## Данные

In [ ]:
!mkdir -p /tmp/taxi
!unzip -o -d /tmp/taxi ./data/taxi.zip

Файл `taxi.parquet` необходимо разбить на несколько частей, чтобы можно было обрабатывать каждую одновременно:

In [ ]:
spark.read.parquet("/tmp/taxi/taxi.parquet").repartition(4).write.mode("overwrite").parquet("/tmp/taxi_many")

# Оптимизатор - Catalyst

Стартовой точкой для запуска вычислений на Apache Spark является запрос (Spark SQL или DataFrame API). Исполнение запроса разбивается на три стадии:

- разбор,
- оптимизация,
- исполнение.

## Разбор запроса

Разбор запроса выполняется в две стадии:

- создание AST - Абстрактного Синтаксического Дерева (Abstract Syntax Tree),
- конвертация AST в реляционное дерево.

![Query to AST](../imgs/catalyst-Query-AST.drawio.svg)

### Абстрактное синтаксическое дерево - AST

Абстрактное синтаксическое дерево (AST) позволяет представить текст запроса или порядок вызова методов DataFrame API в виде дерево для удобства анализа программными средствами.

### Реляционное дерево

Реляционное дерево (relational algebra tree) - это способ визуального представления алгебраических выражений в реляционной алгебре. Основные строительные элементы реляционного дерева:

- узлы для обозначения операций (выборки, проекции и т.д);
- ребра для изображения потоков данных между операциями.

Структура реляционного дерева облегчает анализ и понимание последовательности операций над данными. Трансформация реляционного дерева на базе математических свойств реляционных операций является _**оптимизацией**_ запроса.

> План запроса == реляционное дерево запроса

### Основные понятия реляционной алгебры

Перед продолжением обсуждения оптимизатора и оптимизации запросов необходимо вспомнить основные концепции реляцонной алгебры.

Базовый набор понятий реляционной алгебры включает в себя следующие:

| Название | Обозначение | Описание | Смысл |
|----------|-------------|----------|---------|
| Аттрибут (attribute) | текст  | Единица данных | Колонка в таблице |
| Кортеж (tuple) | N/A | Группа аттрибутов | Строка в таблице |
| Отношение (relation) | Прямоугольник | Коллекция кортежей (строчек) с атрибутами (колонками) | Таблица |
| Проекция (projection) | `π` (пи) | Операция выбора подмножества из множества атрибутов кортежа | Список колонок, которые будут выбраны из таблицы |
| Выборка (selection) | `σ` (сигма) | Операция выбора подмножества кортежей, атрибуты которого попадают по условию _выборки_ | `WHERE` или `HAVING` |
| Объединение (union) | `∪` | Операция, которая на вход получает два отношения и возвращает новое отношение, в котором есть все кортежи исходных отношений. Операция _объединение_ заимствована из теории множеств, а поэтому ожидается, что дубликаты кортежей невозможны в итоговом отношении | `UNION` или `UNION ALL` |
| Пересечение (intersection) | `∩` | Операция, которая на вход получает два отношения и возвращает новое отношение, в котором присутствуют только те кортежи, которые есть в обоих исходных отношениях. Операция _пересечение_ заимствована из теории множеств, а поэтому ожидается, что дубликаты кортежей невозможны в итоговом отношении | `INTERSECT` |
| Разница (difference) | `-` | Операция, которая на вход получает два отношения и возвращает новое отношение, в котором присутствуют только те кортежи первого отношения, которых нет во втором отношении | `EXCEPT` (PostgreSQL) или `MINUS` (Oracle) |
| Соединение (join) | `⨝`, `⟕`, `⟖`, `⟗` | Операция, которая на вход принимает два отношения и двуместный предикат (условие соединения). Результирующее отношение составляет подмножество декартового произведения исходных отношений, для кортежей которых предикат возвращает истинное значение | `JOIN ... ON (...)` |
| Декартово произведение (cartesian product) | `×` | Операция, которая принимает на вход два отношения, а на выходе получается отношение, в котором каждому кортежу из одного исходного отношения ставятся в соответствие все кортежи в другом исходном отношении. Количество кортежей результирующего отношения будет равно произведению количества кортежей в первом и втором исходном отношении | Соединение таблиц без условия |

На практике к результирующему множеству строк могут применяться дополнительные требования. Для этих целей реляционную алгебру расширяют дополнительными операциями:

| Название | Обозначение | Описание | Примеры |
|----------|-------------|----------|---------|
| Cортировка (sorting) | `τ` (тау) | Операция для сортировки множества | `ORDER BY` |
| Переименование (rename) | `ρ` (ро) | Операция замены имени колонки или таблицы | `AS` |
| Аггрегация (aggregation) | `γ` (гамма) | Операция вычисления значения на базе нескольких значений одного столбца | `min`, `max`, `agg`, `sum` |

## Оптимизация запроса

Оптимизация запроса в Apache Spark выполняется при помощи оптимизатора Catalyst. Catalyst включает в себя 4 стадии:

- **Unresolved Logical Plan** - разобранный запрос в виде дерева;
- **Logical Plan** - разобранный запрос, в котором известно на какие объекты из каталога ссылается запрос;
- **Optimized Logical Plan** - запрос, к которому были применены все возможные правила оптимизации;
- **Physical Plan** - запрос в виде RDD DAG для выполнения на кластере Apache Spark.

Объект "Каталог" (Catalog) хранит метаданные о текущем кластере: имена таблиц, UDF и др.

![Catalyst](../imgs/catalyst-stages.drawio.svg)

### Правила оптимизации

Для трансформации дерева запроса применяеются заранее подготовленные правила. Условие применения правила основывается на Pattern Matching: оптимизатор обходит дерево и в каждом узле пытается применить подходящие правила.

Набор правил может выглядеть следующим образом:

    1. Lit(x) * Lit(y) = Lit(y) * Lit(x) = Lit(x * y)
    2. Lit(x) + Lit(y) =  Lit(y) + Lit(x) = Lit(x + y)
    3. Lit(x) / Lit(y) = Lit(x / y)
    4. x * Lit(0) = Lit(0) * x = Lit(0)
    5. x + Lit(0) = Lit(0) + x = x
    6. x * Lit(1) = Lit(1) * x = x
    7. x / Lit(1) = x
    8. x / Lit(0) = Lit(INF)

Результат применения правил может выглядеть следующим образом:

![Catalyst Rules](../imgs/catalyst-rules.drawio.svg)

Правила могут применяться по цепочке:

![Catalyst Rules Chain](../imgs/catalyst-rules-chain.drawio.svg)

Но как оптимизатор понимает, когда нужно остановиться? Есть несколько вариантов:

1. оптимизатор имеет ограничение по времени: не важно сколько правил удалось применить, процесс останавливается через определенный промежуток времени;
2. оптимизатор имеет ограничение на количество примененных правил: оптимизатор должен применить определенное количество правил и не важно сколько времени это займет;
3. снижение стоимости плана не превышает некоторую малую величину $\epsilon$ (эпсилон)
4. ни одно правило больше нельзя применить: функция оптимизатора нашла неподвижную точку.

#### Неподвижная точка - Fixed Point, Fixpoint

**Неподвижной точкой** функции в математике называется такое значение аргумента, при котором значение функции равно значению аргумента. Примеры:

1. для функции `f(x) = x` любое значение аргумента является неподвижной точкой: `f(1) = 1`, `f(2) = 2`, и т.д.;
2. функция `f(x) = x * x` (парабола) имеет две неподвижной точки: 0 (`f(0) = 0 * 0 = 1`) и 1 (`f(1) = 1 * 1 = 1`);
3. для функции `f(x) = x * x - 3 * x + 4` (парабола) неподвижной точкой является значение аргумента равное 2: `f(2) = 2 * 2 - 3 * 2 + 4 = 2`.

Неподвижные точки также называют **инвариантами функции**.

Если рассматривать оптимизатор как функцию `optimize` над множеством деревьев (AST, relational), которая, применяя правила оптимизации, преобразовывает входное дерево и возвращает новое дерево в качестве результата, то можно использовать неподвижную для остановки оптимизатора:

    optimize: tree -> tree

Ниже приведен пример остановки оптимизатора по достижению неподвижной точки (fixpoint). Деревья изображены в виде геометрических фигур для демонстрации того факта, что форма дерева изменяется после запуска функции `optimize`:

![Fixed Point Optimizator](../imgs/catalyst-fixed-point.drawio.svg)

Ниже приведен пример остановки оптимизации AST при помощи неподвижной точки:

![AST Optimizer Stop](../imgs/catalyst-ast-fixpoint.drawio.svg)

### Оптимизированный логический план - Optimized Logical Plan

За один обход дерева можно применить несколько правил оптимизации. Как решить какое из них выбрать? Apache Spark применяет каждое подходящее правило, а в результате появляется несколько деревьев. На следующем шаге уже к каждому новому дереву можно применить другие правила оптимизатора, таким образом количество полученных деревьев растет экспоненциально, и в конечном итоге получается несколько оптимизированных деревьев, которые представляют собой оптимизированный план исполнения запроса.

Ниже на картике можно увидеть процесс оптимизации:

- кружки означают дервья,
- стрелки означают применение одного из правил оптимизации к дереву,
- в самом конце остается список оптимизированных планов.

![Optimized Trees](../imgs/catalyst-optimized-plan.drawio.svg)

### Физический план - Physical Plan

Логические планы оперируют понятиями реляционной алгебры, но у каждого шага логического плана есть физический смысл:

- Relation (отношение, таблица) == FileScan (сканирование файла);
- Projection (проекция, колонка) == Push-down projection (ограничение сканирования файла);
- Aggregate (агрегация) == HashAggregate;
- и т.д.

После получения логических планов, каждому из них сопоставляется физический план, который работает с конкретными реализациями алгоритмов и структур данных и готов для запуска на кластере. В силу того, что Apache Spark кластер может работать только с RDD, получается, что физический план - это RDD. Прямое использование RDD API лишает преимуществ оптимизации, которую дает Catalyst.

> Работа с RDD API напрямую отключает Catalyst и всю оптимизацию в целом!

### Стоимостная модель - Cost Model

Оптимизатор может сгенерировать несколько планов, из которых нужно выбрать наиболее быстрый. Для поиска наиболее быстрого плана используется стоимостная модель (cost model).

Стоимостная модель позволяет оценить на сколько быстро запрос может отработать по указанному плану. Стоимостная модель опирается на статистику. Источником статистики может служить сканируемый файл. Например, `parquet` файл имеет раздел с метаданными из которого можно понять сколько строк хранится в файле, какие есть row groups, размеры страниц и многое другое. Статистику также можно собрать заранее, но использовать ее можно будет только при использовании Spark SQL.

## Анализ плана запроса

### Получение плана запроса

План запроса можно посмотреть двумя способами:

1. DataFrame API,
2. Spark UI.

При этом необходимо использовать оба способа совместно, т.к. каждый из них может предоставить дополнительную информацию, которой нет в другом способе.

### DataFrame API

Для получения плана запроса через DataFrame API необходимо вызвать функцию [DataFrame#explain](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.explain.html) на объекте `DataFrame`:

In [ ]:
taxi_many_part_df = spark.read.parquet("/tmp/taxi_many")
taxi_many_part_df.printSchema()

In [ ]:
taxi_many_part_df \
    .select(F.count(F.lit(1))) \
    .explain()

Вызов метода [DataFrame#explain](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.explain.html) без параметров показывает только физический план, в котором находятся конкретные операции уровня исполнения: сканирование файлов, конкретные реализации функций для агрегатов, сортировки и т.д. Указав дополнительные параметры, можно получить больше информации:

1. `extended` (`False` по умолчанию). Если указать `True`, то можно получить план запроса с каждого шага оптимизатора Catalyst:
    - Unresolved Logical Plan,
    - Resolved Logical Plan,
    - Optimized Logical Plan,
    - Physical Logical Plan.
1. `mode` (`simple` по умолчанию) - указать формат плана запроса:
    - `simple` - напечатать только физический план запроса;
    - `extended` - аналогично `explain(extended=True)`;
    - `codegen` - напечатать `java` код, который будет выполняться, если присутствует стадия `WholeStageCodeGen`;
    - `cost` - показать статистику, если доступно;
    - `formatted` - показать физический план в формате аналогичном Spark UI.

In [ ]:
taxi_many_part_df \
    .select(F.count(F.lit(1))) \
    .explain(mode="cost")

#### Задание

Получить план запроса со статистикой для количества строк с непустым значением `passenger_count`. Сравнить цифры с предыдущим запроса (предыдущая клетка/cell в текщем ноутбуке)

#### Особенности

При помощи DataFrame API можно получить план запроса без запуска самого запроса на исполнение:

In [ ]:
spark.sql("SELECT 'Hello, World!' as greet").explain()

Такая особенность может быть полезной, когда нет желания выполнять большой запрос, чтобы просто посмотреть план запроса. С другой стороны через DataFrame API нельзя получить план запроса для конструкции вида:

    total_rows = taxi_many_part_df.count()

Здесь `taxi_many_part_df` является датафреймом на базе паркетного файла, но при этом [`DataFrame#count`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.count.html) является операцией-действием (action) и немедленно запускает DAG на исполнение. Получить план такого запроса можно только через Spark UI.

### Spark UI

Spark UI позволяет посмотреть план запроса в виде графа, а также подробную статистику по каждой операции. Эта информация может быть полезной при анализе исполнения запроса и поиска традиционных проблем с производительностью, связанных с перекосом (skew) данных.

Недостатком Spark UI является тот факт, что граф запроса может не помещаться на страницу, но план запроса - это SVG картинка, которая может изменять масштаб без потери качества.

Совет:

> Если план запроса очень большой, то рекомендуется уменьшить масштаб страницы до такой степени, чтобы весь план запроса помещался.

Уменьшение масштаба позволяет визуально оценить весь план и заметить некоторые закономерности, повторяющиеся части и т.д. Высокоуровневое понимание запроса позволит эффективнее анализировать запрос по частям в дальнейшем

## Примеры оптимизаций

Примеры оптимизаций будут рассматриваться на базе следующего запроса:

![Relational Tree](../imgs/rel-tree.drawio.svg)

### Pushed Filters

Самым простым, но в то же время исключительно эффективным правилом оптимизации является **Pushed Filter**. Это правило переносит узел `Selection` или `Filter` реляционного дерева на уровень сканирования файла, что позволяет сканировать меньше данных на диске, повышая общую производительность запроса:

![Pushed Filters](../imgs/catalyst-pushed-filters.drawio.svg)

#### Эффективность

Дано: В таблице `emps` 1M строк, 20% (200 000) из которых имеют значение `salary` больше `1000`. В таблице `depts` 500 строк.

Было: В соединении `emps` и `depts` участвовало 1 000 000 * 500 = 500 000 000 строк

Стало: Перед соединением с файловой системы сканируется 200 000 строк, тогда в соединении `emps` и `depts` участвует 200 000 * 500 = 100 000 000 строк. Ускорение в 5 раз!

### Pushed Projections

Подобно сокращению числа сканируемых строк при помощи фильтрации, правило Pushed Projections позволяет сократить количество сканируемых столбцов:

![Pushed Projections](../imgs/catalyst-pushed-projections.drawio.svg)

#### Эффективность

Дано: В таблица `emps` состоит из 1M строк из 50 колонок по 30 байт на значение в среднем, 20% (200 000) из которых имеют значение `salary` больше `1000`. В таблице `depts` 500 строк и 10 колонок по 30 байт на значение в среднем.

**Было**: В соединении `emps` и `depts` участвовало:

    - со стороны `emps` 200 000 строк по 50 колонок по 30 байт на колонку => 200 000 * 50 * 30 = 300 000 000 байт = 300 МБ;
    - со стороны `depts` 500 строк по 10 колонок по 30 байт на колонку => 500 * 10 * 30 = 150 000 байт = 150 КБ.

**Стало**: Перед соединением `emps` и `depts` сканируется:

    - со стороны `emps` 200 000 строк по 3 колонки по 30 байт на колонку => 200 000 * 3 * 30 = 18 000 000 байт = 18 МБ => Снижение трафика в 16 раз (300 / 18);
    - со стороны `depts` 500 строк по 2 колонки по 30 байт на колонку => 500 * 2 * 30 = 30 000 байт = 30 КБ => Снижение трафика в 5 раз.

### Демонстрация

Подготовить датафрейм:

In [ ]:
taxi_many_part_df = spark.read.parquet("/tmp/taxi_many")
taxi_many_part_df.printSchema()

На базе низкокардинальной колонки `PULocationID` создается новый датафрейм для местоположений:

In [ ]:
locations_df = (
    taxi_many_part_df
        .select(col("PULocationID").alias("id"))
        .distinct()
        .withColumn("name", F.rand())
)

locations_df.write.mode("overwrite").parquet("/tmp/locations")

locations_df = spark.read.parquet("/tmp/locations")

Соединение датафреймов `taxi_many_part_df` и `locations_df` демонстрирует применение правил **Pushed filters** и **Pushed Projections**:

In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

result_df = (
    taxi_many_part_df
        .join(locations_df, col("PULocationID") == col("id"))
        .where("passenger_count > 5")
        .orderBy("name")
        .select("VendorID", "trip_distance", "name")
)
result_df.explain()

На плане видно, что выполнились оба правила оптимизации: Pushed Filters и Pushed Projections. Дополнительно была использована цветовая индикация для подсветки колонок: одна колонка == один цвет

![Catalyst Rules Demo](../imgs/catalyst-rules-demo.jpg)

## Адаптивный оптимизатор запросов - Adaptive Query Execution

Адаптивный оптимизатор запросов (Adaptive Query Execution, AQE) собирает статистику во время выполнения запроса и может изменять его план, если, согласно статистике, найден более эффективный способ исполнения запроса.

Адаптивный оптимизатор запросов появился в Apache Spark 3.0 и активирован по умолчанию. Чтобы проверить, что AQE включен, необходимо проверить значение опции `spark.sql.adaptive.enabled`:

In [ ]:
active = "Включен" if spark.conf.get("spark.sql.adaptive.enabled") == "true" else "Выключен"

print(f"Адаптивный оптимизатор запросов: {active}")

В AQE входят три оптимизации:

- замена стратегии соединений таблиц (join) на Broadcast JOIN (Converting sort-merge join to broadcast join),
- разбиение перекошенных (skew) партиций (Optimizing Skew Join),
- объединение партиций небольшого размера (Coalescing Post Shuffle Partitions).

Настройка `spark.sql.adaptive.enabled` является "зонтиком" над этими тремя оптимизациями. Если `spark.sql.adaptive.enabled` отключена, то ни одна оптимизация AQE не будет доступна.

### Замена стратегии соединения таблиц на Broadcast JOIN

Apache Spark по умолчанию выбирает Sort Merge Join в качестве стратегии соединения (join) таблиц по умолчанию, т.к. Sort Merge Join позволяет гарантировано обработать данные любого размера. Sort Merge Join не может похвастаться высокой производительностью. Стратегия Broadcast Join является самой быстрой стратегией соединения таблиц, но при этом она более требовательна к оперативной памяти: необходимо один из датафреймов полностью скопировать в память каждого воркера.

#### Условия активации Broadcast JOIN

Для активации Broadcast JOIN необходимо указать пороговое значение в байтах через настройку `spark.sql.autoBroadcastJoinThreshold`:

In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10 * 1024 * 1024)

Значением по умолчанию является 10МБ.

### Разбиение перекошенных партиций

Неравномерное распределение данных по партициям является одной из самых больших проблем при работе с Apache Spark. Ниже приведен пример датафрейма, в котором одна из партиций на много больше остальных:

![](../imgs/catalyst-skewed-partitions.drawio.svg)

Обработка подобного датафрейма будет тормозить процесс, потому что стадия (stage) не может завершиться, пока не завершены все задания (task) в стадии. Ниже приведен пример обработки датафрейма с перекосом данных по партициям:

![](../imgs/catalyst-skewed-partitions-processing.drawio.svg)

Из картики видно, что два воркера простаивали большую часть времени, т.к. необходимо было дождаться завершения обработки задачи по партиции `P1`.

AQE Optimizing Skew Join оптимизация может автоматически разбить большие партиции на партиции меньшего размера, увеличивая таким образом скорость обработки запроса:

![](../imgs/catalyst-aqe-skew-partitions.drawio.svg)

Но работать эта оптимизация будет только во время соединений таблиц (JOIN): AQE Skew Join не используется, если в запросе нет JOIN

> **AQE Skewed join работает только с JOIN запросами**

#### Условия активации

По умолчанию AQE Optimizing Skew Join оптимизация включена. Дополнительно можно проверить значение настройки `spark.sql.adaptive.skewJoin.enabled`:

In [ ]:
active = "Включен" if spark.conf.get("spark.sql.adaptive.skewJoin.enabled") == "true" else "Выключен"

print(f"AQE Optimizing Skew Join: {active}")

Не любая большая партция может считаться перекошенной, для этого необходимо чтобы выплнялись два условия:

- партиция больше среднего размера остальных партиций в `spark.sql.adaptive.skewJoin.skewedPartitionFactor` раз (по умолчанию в 5 раз),
- партиция превышает по размеру значение `spark.sql.adaptive.skewJoin.skewedPartitionThresholdInBytes` (по умолчанию в 256МБ).

AQE Optimizing Skewed Join может не активироваться, т.к. она привносит дополнительную операцию перемешивания (Shuffle), а Catalyst может найти более быстрый план.

Для принудительной активации оптимизации AQE Optimizing Skewed Join в Apache Spark 3.3.0 появилась настройка `spark.sql.adaptive.forceOptimizeSkewedJoin`.

### Объединение партиций небольшого размера

Apache Spark создает `spark.sql.shuffle.partitions` партиций после перемешивания (shuffle), но может так получиться, что какие-то из партиций получаются очень маленького размера. AQE может автоматически объединить их при помощи **Coalescing Post Shuffling Partitions** (без дополнительных Shuffle операций).

![](../imgs/catalyst-post-shuffle-coalesce.drawio.svg)

#### Условия активации

По умолчанию **AQE Coalescing Post Shuffle Partitions** оптимизация включена. Дополнительно можно проверить значение настройки `spark.sql.adaptive.coalescePartitions.enabled`:

In [ ]:
active = "Включен" if spark.conf.get("spark.sql.adaptive.coalescePartitions.enabled") == "true" else "Выключен"

print(f"AQE Coalescing Post Shuffle Partitions: {active}")

In [ ]:
sc.setJobDescription("Сгенерировать датафреймы")

source = spark.range(1, 10000, 1, 10).toDF("id")
left = source.select(F.expr("id % 3").alias("id"), col("id").alias("value")) 
right = source.select(F.lit(0).alias("id"), col("id").alias("value"))

In [ ]:
left.write.mode("overwrite").parquet("/tmp/left")
right.write.mode("overwrite").parquet("/tmp/right")

In [ ]:
left = spark.read.parquet("/tmp/left")
right = spark.read.parquet("/tmp/right")

In [ ]:
sc.setJobDescription("Получить распределение по партициям в `left`")

left.groupBy(F.spark_partition_id()).count().show()

In [ ]:
sc.setJobDescription("Получить распределение по партициям в `right`")

right.groupBy(F.spark_partition_id()).count().show()

In [ ]:
sc.setJobDescription("Соединить `left` и `right`")

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", True)
spark.conf.set("spark.sql.shuffle.partitions", 77)

left.join(right, "id").count()

In [ ]:
sc.setJobDescription("Соединить `left` и `right`: Без автоматического слияния партций")

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)
spark.conf.set("spark.sql.shuffle.partitions", 77)

left.join(right, "id").count()

На плане видно:

1. оптимизатор добавил дополнительный узел `AQEShuffleRead` в план с автоматическим слиянием партиций,
2. После `AQEShuffleRead` количество партиций снизилось до 1 в плане с автоматическим слиянием партиций.


![](../imgs/catalyst-post-shuffle-coalesce-plan.drawio.svg)

#### Задание

1. Сколько заданий (task) было выполнено в запросе с AQE Coalescing Post Shuffle Partitions?
1. Сколько заданий (task) было выполнено в запросе без AQE Coalescing Post Shuffle Partitions?